Lets's first get an idea of the size of the data set. The file is neaerly 400 MB in size. Let's find out how many of each tags exist within the dataset.

In [1]:
import xml.etree.cElementTree as ET

osmfile = "wlinn"

def tag_count(filename):
    
    """
    Counts how many tags are within the XML file and returns it as a dictionary
    """
    
    tag_dictionary = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tag_dictionary:
            tag_dictionary[elem.tag] = 1
        else:
            tag_dictionary[elem.tag] += 1
            
    return tag_dictionary

print (tag_count(osmfile))

{'note': 1, 'meta': 1, 'bounds': 1, 'node': 1643082, 'tag': 1180922, 'nd': 1898598, 'way': 223390, 'member': 50651, 'relation': 1998, 'osm': 1}


Here is a summary of what our function outputs:

- 134,382 nodes
- 223,390 ways
- 1,998 relations
- 50,651 members
- 1,898 nds

That's quite a lot to work with. But it's definitely not impossible. Let's continue.

Let's discover which values in the tags will give us issues. Not all characters can be imported easily into a database and having consistency will enhance readability. It will be much easier later on to have consistency in our data. We are going to be using regular expresssions (REGEX) in Python. This is what we want to identify in our OSM File:

- Values with only lower case letters
- Values with only upper case letters
- Values that have characters that you wouldn't expect to be in a map
    
Let's handle this in the function below

In [2]:
import re

lowercase = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
uppercase = re.compile(r'^([A-Z]|_)*$')
upper_colon = re.compile(r'^([A-Z]|_)*:([a-z]|_)*$')
problem = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

"""
    This funtion will create a dictionary telling us how many entries in the
    dataset contain all lower case for "k=" values, all uppercase, if the "k="
    value contains any problem characters, values with either all lower or all
    upper with at least 1 colon, or if there are any characters not
    convered in the REGEX
"""

def k_type(element, key):
            
        if element.tag == 'tag': #find only elements named tag
            
            if lowercase.search(element.attrib['k']): #finds the "k" value in the tag
                key['lowercase'] +=1
            elif lower_colon.search(element.attrib['k']):
                key['lower_colon'] +=1
            elif uppercase.search(element.attrib['k']):
                key['uppercase'] +=1
            elif upper_colon.search(element.attrib['k']):
                key['upper_colon'] +=1
            elif problem.search(element.attrib['k']):
                key['problem'] += 1
            else:
                key['other'] += 1
        return key


"""This fuction will parse through an XML file (the OSM file) and will
    execute the above function to count the different types of k values 
    that we have.
"""

def process_tag(filename):
    
    # sets the key variable with 0 in all indexes
    key = {"lowercase": 0, "lower_colon": 0, "uppercase": 0, "upper_colon": 0, "problem": 0, "other": 0} 
    
    for _, element in ET.iterparse(filename):
        key = k_type(element, key)
        
    return key

tag_dictionary = process_tag(osmfile)
print (tag_dictionary)

{'lowercase': 617224, 'lower_colon': 557043, 'uppercase': 965, 'upper_colon': 2876, 'problem': 0, 'other': 2814}


Thankfully we haven't assesed anything that I would consider a problem in this dataset. However, we do have 2876 tags that do fit the "uppercase" description. Let's make a function that takes the dataset as an input and have it output the tags' k value associated with it. We will see what corresponds with the "uppercase" that our above function has identified.

In [3]:
def get_key_with_issues(filename):
    
    
    #takes filename and returns a list of identified issues

    issue_list = []
    for _, element in ET.iterparse(filename):
        if element.tag == 'tag':
            if uppercase.search(element.attrib['k']):
                issue_list.append(element.attrib['k'])
    return issue_list

list = get_key_with_issues(osmfile)
list[:10]

['NHS', 'NHS', 'NHS', 'NHS', 'NHS', 'NHS', 'NHS', 'NHS', 'NHS', 'NHS']

I truncated the list with list[:100], but there are a lot more results. This is a lot of NHS. Upon some further research, the "NHS" value was put in by one overzealous user named Peter Dobratz in 2016. NHS in this case stands for "National Highway System." This is acceptable to have and doesn't necessarily make sense to spell out the acronym in all 2876 cases. It seems our script has caught an erroneous error. I will leave this data untouched.

### Cleaning and Auditing the Data

We will analyze the street types that are in this dataset and try to get an angle on how we want to find issues that arise.

In [4]:
from collections import defaultdict

#we are using defaultdict incase we access a key that doesn't exist yet

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def check_street_types(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] += 1

def print_sorted_dictionary(d, expresssion):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print (expression % (k, v))
        
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def check(filename):
    for event, elem in ET.iterparse(filename):
        if is_street_name(elem):
            check_street_types(street_types, elem.attrib['v'])
    print(street_types, "%s: %d")
    return(street_types)
    
every_type = check(osmfile)
every_type

defaultdict(<class 'int'>, {'Court': 13102, 'Road': 17588, 'Street': 26807, 'Drive': 19373, 'Rd': 7, 'Way': 4571, 'Boulevard': 2795, 'Lane': 6922, 'Avenue': 24919, 'East': 42, 'Circle': 2026, 'Highway': 504, 'Place': 3002, 'West': 72, 'Loop': 1431, 'Terrace': 1362, 'Alley': 2, '213': 107, 'Cervantes': 53, 'Summit': 25, 'Circus': 30, '212': 120, '224': 48, 'Parkway': 367, '97266': 1, 'Ave': 2, 'North': 44, 'Landing': 9, 'Botticelli': 7, 'Touchstone': 55, 'Point': 15, 'South': 47, '99E': 41, 'Vista': 4, 'Wheatland': 4, 'Run': 21, 'Crest': 42, 'Pointe': 2, 'Trail': 170, 'Grotto': 4, 'Downs': 29, 'Polonius': 5, 'Falstaff': 12, 'Pimlico': 4, 'Wheatherstone': 2, 'Woods': 15, 'Hotspur': 12, 'Greco': 1, 'Curve': 11, 'Path': 13, 'Miami': 17, 'Northbound': 1, 'Southbound': 1, 'Spinosa': 20, 'Pericles': 6, 'Commons': 37, 'View': 27, 'Fieldcrest': 46, 'TRL': 2}) %s: %d


defaultdict(int,
            {'Court': 13102,
             'Road': 17588,
             'Street': 26807,
             'Drive': 19373,
             'Rd': 7,
             'Way': 4571,
             'Boulevard': 2795,
             'Lane': 6922,
             'Avenue': 24919,
             'East': 42,
             'Circle': 2026,
             'Highway': 504,
             'Place': 3002,
             'West': 72,
             'Loop': 1431,
             'Terrace': 1362,
             'Alley': 2,
             '213': 107,
             'Cervantes': 53,
             'Summit': 25,
             'Circus': 30,
             '212': 120,
             '224': 48,
             'Parkway': 367,
             '97266': 1,
             'Ave': 2,
             'North': 44,
             'Landing': 9,
             'Botticelli': 7,
             'Touchstone': 55,
             'Point': 15,
             'South': 47,
             '99E': 41,
             'Vista': 4,
             'Wheatland': 4,
             'Run': 21,
         

There are quite a few issues here. 

- Roads are erroneously labeled as "Rd" and "Ave" should be spelt out.
- 212, 224, 213, and 99E are all highways in the state of Oregon. They should be labeled a bit differently and not just by their number.
- 97266 is an area/zip code and should not be in the street name field.
- Pimlico is the name of a street in my hometown and should be appended with "Street" to aid in consistency.
- The value "TRL" should be spelt out fully as "Trail" to aid in consistency as well.

We will fix these items.

In [5]:
from collections import defaultdict

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] += 1

def print_sorted_dict(d, expression):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print (expression % (k, v))

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit(filename):
    for event, elem in ET.iterparse(filename):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print(street_types, "%s: %d")
    return(street_types)

all_types = audit(osmfile)
all_types

defaultdict(<class 'int'>, {'Court': 13102, 'Road': 17588, 'Street': 26807, 'Drive': 19373, 'Rd': 7, 'Way': 4571, 'Boulevard': 2795, 'Lane': 6922, 'Avenue': 24919, 'East': 42, 'Circle': 2026, 'Highway': 504, 'Place': 3002, 'West': 72, 'Loop': 1431, 'Terrace': 1362, 'Alley': 2, '213': 107, 'Cervantes': 53, 'Summit': 25, 'Circus': 30, '212': 120, '224': 48, 'Parkway': 367, '97266': 1, 'Ave': 2, 'North': 44, 'Landing': 9, 'Botticelli': 7, 'Touchstone': 55, 'Point': 15, 'South': 47, '99E': 41, 'Vista': 4, 'Wheatland': 4, 'Run': 21, 'Crest': 42, 'Pointe': 2, 'Trail': 170, 'Grotto': 4, 'Downs': 29, 'Polonius': 5, 'Falstaff': 12, 'Pimlico': 4, 'Wheatherstone': 2, 'Woods': 15, 'Hotspur': 12, 'Greco': 1, 'Curve': 11, 'Path': 13, 'Miami': 17, 'Northbound': 1, 'Southbound': 1, 'Spinosa': 20, 'Pericles': 6, 'Commons': 37, 'View': 27, 'Fieldcrest': 46, 'TRL': 2}) %s: %d


defaultdict(int,
            {'Court': 13102,
             'Road': 17588,
             'Street': 26807,
             'Drive': 19373,
             'Rd': 7,
             'Way': 4571,
             'Boulevard': 2795,
             'Lane': 6922,
             'Avenue': 24919,
             'East': 42,
             'Circle': 2026,
             'Highway': 504,
             'Place': 3002,
             'West': 72,
             'Loop': 1431,
             'Terrace': 1362,
             'Alley': 2,
             '213': 107,
             'Cervantes': 53,
             'Summit': 25,
             'Circus': 30,
             '212': 120,
             '224': 48,
             'Parkway': 367,
             '97266': 1,
             'Ave': 2,
             'North': 44,
             'Landing': 9,
             'Botticelli': 7,
             'Touchstone': 55,
             'Point': 15,
             'South': 47,
             '99E': 41,
             'Vista': 4,
             'Wheatland': 4,
             'Run': 21,
         

##### 1 We will make a dictionary of what we should expect

In [6]:
expected_values = ['Avenue', 'Alley', 'Road', 'Street', 'Trail', 'Landing', 'Pointe', 
                   'Vista', 'Woods', 'Curve', 'Path', 'Freeway', 'Grotto', 'Court', 
                   'Northbound', 'Southbound', 'Drive', 'Boulevard', 'Lane', 'Circle',
                   'Highway', 'Place', 'Loop', 'Terrace', 'Way', 'Crest', 'Parkway',
                   'Point']

abbr_mapping = {'Ave': 'Avenue',
                'TRL': 'Trail',
                'Hwy': 'Highway',
                'Rd': 'Road',
                'Ave': 'Avenue',
                'Ct': 'Court',
                'Dr': 'Drive',
                'Pl': 'Place',
                'place': 'Place',
                'Pkwy': 'Parkway',
                'rd.': 'Road',
                'Sq.': 'Square',
                'St': 'Street',
                'st': 'Street',
                'ST': 'Street',
                'St,': 'Street',
                'St.': 'Street',
                'street': 'Street',
                'Street.': 'Street'
                }

In the below code, we are only going to be printing values where the type is <20 and that are not yet in expected_values.

In [7]:
typo_full_names = {}

def audit_street_name(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if (all_types[street_type] < 20) and (street_type not in expected_values) and (street_type not in abbr_mapping):
            if street_type in typo_full_names:
                typo_full_names[street_type].append(street_name)
            else:
                typo_full_names.update({ street_type:[street_name] })

def audit_name(filename):
    for event, elem in ET.iterparse(filename):
        if is_street_name(elem):
            audit_street_name(street_types, elem.attrib['v'])    
    return typo_full_names

audit_name(osmfile)

{'97266': ['8202 SE Flavel St, Portland, OR 97266'],
 'Botticelli': ['Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli'],
 'Wheatland': ['Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland'],
 'Polonius': ['Polonius', 'Polonius', 'Polonius', 'Polonius', 'Polonius'],
 'Falstaff': ['Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff'],
 'Pimlico': ['Pimlico', 'Pimlico', 'Pimlico', 'Pimlico'],
 'Wheatherstone': ['Wheatherstone', 'Wheatherstone'],
 'Hotspur': ['Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur',
  'Hotspur'],
 'Greco': ['El Greco'],
 'Miami': ['Southwest Miami',
  'Southwest Miami',
  'Southwest Miami',
  'Southwest Miami',
  'Southwest Miami',
  'Southwest Miami',
  'Southwes

Using the above output, we have a lot of work to do.

1. Boticelli is a street.
2. 97266 is a ZIP code and should not be in this field. The location refers to a delicious Mexican restaurant. The v field in the tag element is "8202 SE Flavel St, Portland, OR 97266". It should just be "SE Flavel Street".
3. Wheatland is a road.
4. Falstaff is a road.
5. Pimlico is a Drive.
6. Hotspur is a road.
7. Southwest Miami is a street.
8. Pericles is a loop.
9. Polonius is a loop.
10. El Greco is a street.
11. Wheatherstone is a street
13. View, Commons, Run, South, North, Circus, Summit, Downs, West, View, and East are all acceptable values. I will add them to the expected_values dictionary.
14. Upon further inspection, Highways 99E, 213, 212, and 224 are all labeled correctly. They will be added to expected_values as well
15. Cervantes is a street.
16. Touchstone is a road.
17. Polonius is a street.
18. Spinosa is a road.
19. Southeast Fieldcrest is a road.

Let's get to work.

In [8]:
expected_values.extend([
    'View', 'Commons', 'Run', 'South', 'North', 'East', 'Circus', 'Summit', 'West', 
    '99E', '224', '213', 'View', '212', 'Downs'
                       ])

Let's rerun the above script to get a final output of what to fix:

In [9]:
audit_name(osmfile)

{'97266': ['8202 SE Flavel St, Portland, OR 97266',
  '8202 SE Flavel St, Portland, OR 97266'],
 'Botticelli': ['Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli',
  'Botticelli'],
 'Wheatland': ['Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland',
  'Southwest Wheatland'],
 'Polonius': ['Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius',
  'Polonius'],
 'Falstaff': ['Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Falstaff',
  'Fal

That's much better and much more clear. Let's put the "broken" values into a dictionary so that we can manipulate them properly.

In [10]:
name_fix = { 
                'Boticelli': 'Boticelli Street',
                'Southwest Wheatland': 'Southwest Wheatland Road',
                'Falstaff': 'Falstaff Road',
                'Pimlico': 'Pimlico Drive',
                'Hotspur': 'Hotspur Road',
                'Southwest Miami': 'Southwest Miami Street',
                'Pericles': 'Pericles Loop',
                'Polonius': 'Polonius Loop',
                'El Greco': 'El Greco Street',
                'Wheatherstone': 'Wheatherstone Street',
                '8202 SE Flavel St, Portland, OR 97266': 'SE Flavel Street',
                'Cervantes': 'Cervantes Street',
                'Touchstone': 'Touchstone Road',
                'Polonius': 'Polonius Street',
                'Spinosa': 'Spinosa Road',
                'Southeast Fieldcrest': 'Southeast Fieldcrest Road'
               }

spelling_fix = { 
                'Falstaff': 'Falstaff Road',
                'Pimlico': 'Pimlico Drive',
                'Hotspur': 'Hotspur Road',
                'Pericles': 'Pericles Loop',
                'Polonius': 'Polonius Loop',
                'El Greco': 'El Greco Street',
                '8202 SE Flavel St, Portland, OR 97266': 'SE Flavel Street',
                'Cervantes': 'Cervantes Street',
                'Touchstone': 'Touchstone Road',
                'Polonius': 'Polonius Street',
                'Spinosa': 'Spinosa Road',
               }

# Let's sort our expected variable to aid in readability

expected_values = sorted(expected_values)
expected_values

['212',
 '213',
 '224',
 '99E',
 'Alley',
 'Avenue',
 'Boulevard',
 'Circle',
 'Circus',
 'Commons',
 'Court',
 'Crest',
 'Curve',
 'Downs',
 'Drive',
 'East',
 'Freeway',
 'Grotto',
 'Highway',
 'Landing',
 'Lane',
 'Loop',
 'North',
 'Northbound',
 'Parkway',
 'Path',
 'Place',
 'Point',
 'Pointe',
 'Road',
 'Run',
 'South',
 'Southbound',
 'Street',
 'Summit',
 'Terrace',
 'Trail',
 'View',
 'View',
 'Vista',
 'Way',
 'West',
 'Woods']

### Now We Will Clean The Data!
##### (This is the best part!)

The name_fix dictionary was created. Now we can actually put it to use.

In [16]:
def update_name(name):
    street_type = name.split(' ')[-1]
    street_name = name.rsplit(' ', 1)[0]
    if street_type in abbr_mapping:
        name = street_name + ' ' + abbr_mapping[street_type]
    elif street_type in spelling_fix:
        if 'Falstaff' in street_name:
            name = 'Falstaff Road'
        elif 'Pimlico' in street_name:
            name = 'Pimlico Drive'
        elif 'Hotspur' in street_name:
            name = 'Hostspur Road'
        elif 'Pericles' in street_name:
            name = 'Pericles Loop'
        elif 'Polonius' in street_name:
            name = 'Polonius Loop'
        elif 'El Greco' in street_name:
            name = 'El Greco Street'
        elif '8202 SE Flavel St, Portland, OR 97266' in street_name:
            name = 'SE Flavel Street'
        elif 'Cervantes' in street_name:
            name = 'Cervantes Street'
        elif 'Touchstone' in street_name:
            name = 'Touchstone Road'
        elif 'Spinosa' in street_name:
            name = 'Spinosa Road'            
    return name    

def audit_abbreviations(filename):
    problem_street_types = defaultdict(set)
    for event, elem in ET.iterparse(filename):
        if is_street_name(elem):
            expected_street_type(problem_street_types, elem.attrib['v'])
    return problem_street_types

def expected_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected_values:
            street_types[street_type].add(street_name)

def run_updates(filename):
    st_types = audit_abbreviations(osmfile)
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name)
            if better_name != name:
                corrected_names[name] = better_name
    return corrected_names
            
corrected_names = {}           
corrected_names = run_updates(osmfile)
print_sorted_dict(corrected_names, "%s: %s")

4th Ave: 4th Avenue
7273 SE 92nd Ave: 7273 SE 92nd Avenue
Cervantes: Cervantes Street
Falstaff: Falstaff Road
Hotspur: Hostspur Road
Pericles: Pericles Loop
Pimlico: Pimlico Drive
Polonius: Polonius Loop
S Carus Rd: S Carus Road
S Penman Rd: S Penman Road
SE Stevens Rd: SE Stevens Road
SE Sunnyside Rd: SE Sunnyside Road
Southeast Hittay TRL: Southeast Hittay Trail
Southwest Borland Rd: Southwest Borland Road
Spinosa: Spinosa Road
SW Boones Ferry Rd: SW Boones Ferry Road
Touchstone: Touchstone Road


In [17]:
theschema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [20]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

OSM_PATH = "osmfile"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def correct_element(v):
    if v in corrected_names:
        correct_value = corrected_names[v]
    else:
        correct_value = v
    return correct_value

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['user'] = element.attrib['user']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']
        
        for node in element:
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if ':' in node.attrib['k']:
                tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                tag_dict['value'] = correct_element(node.attrib['v'])
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = node.attrib['k']
                tag_dict['value'] = correct_element(node.attrib['v'])
            tags.append(tag_dict)
            
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['version'] = element.attrib['version']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']
        n = 0
        for node in element:
            if node.tag == 'nd':
                way_dict = {}
                way_dict['id'] = element.attrib['id']
                way_dict['node_id'] = node.attrib['ref']
                way_dict['position'] = n
                n += 1
                way_nodes.append(way_dict)
            if node.tag == 'tag':
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                if ':' in node.attrib['k']:
                    tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                    tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                    tag_dict['value'] = correct_element(node.attrib['v'])
                else:
                    tag_dict['type'] = 'regular'
                    tag_dict['key'] = node.attrib['k']
                    tag_dict['value'] = correct_element(node.attrib['v'])
                tags.append(tag_dict)
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=theschema):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('ascii') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', encoding="ascii") as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'w', encoding="ascii") as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w', encoding="ascii") as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w', encoding="ascii") as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w', encoding="ascii") as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow((el['way']))
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
process_map(osmfile, validate=True)

KeyboardInterrupt: 